In [11]:
#[derive(Clone, Copy)]
enum DiffNode<'a>{
    Add(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Sub(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Mul(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Div(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Value(f32),
    WithSymbol(&'a DiffNode<'a>, &'static str),
}
use DiffNode::*;

impl<'a> DiffNode<'a>{
    fn has_symbol(&'a self, symbol: &'static str) -> bool{
        if let WithSymbol(_, sym) = self{
            sym==&symbol
        }else{false}
    }

    fn eval(&'a self) -> f32{
        match self{
            Add(a, b) => a.eval() + b.eval(),
            Sub(a, b) => a.eval() - b.eval(),
            Mul(a, b) => a.eval() * b.eval(),
            Div(a, b) => a.eval() / b.eval(),
            Value(v) => *v,
            WithSymbol(n, _) => n.eval()
        }
    }
    
    fn derivative(&'a self, x: &'static str) -> f32{
        match self{
            Add(a, b) => a.derivative(x) + b.derivative(x),
            Sub(a, b) => a.derivative(x) + -b.derivative(x),
            Mul(a, b) => a.derivative(x) * b.eval() + b.derivative(x) * a.eval(),
            
            // WRONG!
            Div(a, b) => 1./b.eval() * -a.eval()/b.eval(),
            
            Value(_) => 0.,
            WithSymbol(inner, sym) => if sym == &x{1.}else{inner.derivative(sym)}
        }
    }
    
    fn with_symbol(&'a self, sym: &'static str) -> Self{
        WithSymbol(self, sym)
    }
}

In [12]:
let x = Value(2.).with_symbol("x");

let y = Sub(&Add(&Value(1.), &Mul(&Add(&x, &Value(2.)), &Add(&Value(3.), &Value(3.)))), &x).derivative("x");

println!("{y}");

5
